In [107]:
##LIBRERIAS##
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.constants import k 

In [108]:
##CONSTANTES##
#Relaciones de cantidades
C = 100; r = 10; G = 50; E = 10; nanodomin_space = 10 

type_properties = {
            "G*": {"color":"#3498DB" , "ratio": 0.15, "active": True, "mobile": True},
            "G_inact": {"color": "#2ECC71", "ratio": 0.15, "active": False, "mobile": True},
            "PDE*": {"color": "#F1C40F", "ratio": 0.25, "active": True, "mobile": True},
            "PDE_inact": {"color": "#F39C12", "ratio": 0.25, "active": False, "mobile": True},
            "Rh*": {"color": "#7D3C98", "ratio": 0.12, "active": True, "mobile": True},
            "GMPc": {"color":"#E74C3C" , "ratio":0.08, "active": False, "mobile": True}
        }


tau_species = {
    "Rh*": 0.040,     # 40 ms
    "G*": 0.010,      # 10 ms
    "PDE*": 0.020     # 20 ms
}

In [109]:
#CREACIÓN DE MOLÉCULAS#
class Molecule:
    def __init__(self, name, pos, vel, color, ratio_molecular, energy, mass, 
                 active=False, mobile=True, temperature=300.0):
        self.name = name         
        self.pos = np.array(pos, dtype=float)   
        self.vel = np.array(vel, dtype=float)   
        self.color = color       
        self.ratio = float(ratio_molecular)  
        self.energy = float(energy)         
        self.active = active     
        self.m = float(mass)
        self.mobile = mobile
        self.collision_count = 0
        self.temperature = temperature  
        
    def deactivate_step(self, dt):
        if self.name in tau_species:
            if np.random.rand() < dt / tau_species[self.name]:
                # versión inactiva correspondiente
                if self.name == "Rh*":
                    self.name = "Rh_inact"
                elif self.name == "G*":
                    self.name = "G_inact"
                elif self.name == "PDE*":
                    self.name = "PDE_inact"
                self.active = False
            
    def set_temperature(self, temp):
        """Establece la temperatura y ajusta la velocidad correspondientemente"""
        self.temperature = temp
        self.update_speed_from_temperature()
    
    def update_speed_from_temperature(self):
        """Actualiza la velocidad basándose en la temperatura usando teoría cinética"""
        if self.temperature <= 0 or not self.mobile:
            self.vel = np.array([0.0, 0.0])
            self.energy = 0.0
            return
        
        mean_speed = np.sqrt(8 * k * self.temperature / (np.pi * self.m)) # Velocidad promedio según teoría cinética
        angle = np.random.uniform(0, 2 * np.pi)
        self.vel = np.array([mean_speed * np.cos(angle), mean_speed * np.sin(angle)])
        self.update_energy_from_speed()

    def update_speed_from_energy(self):
        """Recalcula velocidad desde energía (mantiene dirección)"""
        if self.energy <= 0:
            self.vel = np.array([0.0, 0.0]) 
            return
            
        speed = np.sqrt(2.0 * self.energy / self.m)
        
        current_speed = np.linalg.norm(self.vel) #Calcula la magnitud de la velocidad u-u 
        
        if current_speed > 1e-9:
            direction = self.vel / current_speed  
            self.vel = direction * speed         
            
        else: #Si la velocidad es muy pequeña me invento que vaya a un lugar super aleatorio en velocidad
            angle = np.random.uniform(0, 2 * np.pi)
            self.vel = np.array([speed * np.cos(angle), speed * np.sin(angle)])

    def update_energy_from_speed(self):
        """Recalcula energía desde velocidad actual"""
        speed = np.linalg.norm(self.vel)
        self.energy = 0.5 * self.m * (speed ** 2)


    def move(self, L, dt=0.1):
        if not self.mobile:
            return  

        if self.name == "Rh*":
            D = 0.5
        elif self.name == "GMPc":
            D = 1.5
        elif self.name == "G*" or self.name == "G_inact":
            D = 0.8
        elif self.name == "PDE*" or self.name == "PDE_inact":
            D = 0.6
        else:
            D = 0.05
        
        # Desplazamiento browniano 
        angle = np.random.uniform(0, 2 * np.pi)
        brownian_displacement = np.sqrt(2 * D * dt) * np.array([np.cos(angle), np.sin(angle)]) #Este D es de la constante de difusión 
        new_pos = self.pos + brownian_displacement
        
        # CONDICIONES TOROIDALES. Si sale por un lado, entra por el opuesto
        new_pos[0] = new_pos[0] % L  
        new_pos[1] = new_pos[1] % L 
        
        self.pos = new_pos


    def handle_reaction_collision(self, other, system):
        """Maneja colisión con reacción química CON probabilidades"""
        
        reaction_probabilities = {
            "rh_g": 1,    # Rh* + G_inact -> G* (baja por complejidad estérica)
            "g_pde": 0.5, # G* + PDE_inact -> PDE* (aún más compleja) 
        }
        
        reaction_type = None
        success = False
        
        # Rh* + G_inact -> G*
        if (self.name == "Rh*" and other.name == "G_inact") or (other.name == "Rh*" and self.name == "G_inact"):
            reaction_type = "rh_g"
            success = np.random.random() < reaction_probabilities["rh_g"]
            
        # G* + PDE_inact -> PDE*
        elif (self.name == "G*" and other.name == "PDE_inact") or (other.name == "G*" and self.name == "PDE_inact"):
            reaction_type = "g_pde" 
            success = np.random.random() < reaction_probabilities["g_pde"]
        
        
        if success and reaction_type:
            if reaction_type == "rh_g":
                if self.name == "G_inact":
                    self.convert_to("G*", system)
                else:
                    other.convert_to("G*", system)
            elif reaction_type == "g_pde":
                if self.name == "PDE_inact":
                    self.convert_to("PDE*", system)  
                else:
                    other.convert_to("PDE*", system)
    
        return reaction_type

    def handle_physical_collision(self, other, system):
        """Maneja colisión física elástica la idea es que cuando colisiones con otras partículas
        rebote hacia otro lugar de forma más o menos v - 2 v * (pos/distance)""" 
        if not self.mobile and not other.mobile:
            return
            
        delta_pos = other.pos - self.pos
        distance = np.linalg.norm(delta_pos)
        
        if distance < 1e-12:
            return
            
        normal = delta_pos / distance
        
        if self.mobile and not other.mobile:
            v_dot_n = np.dot(self.vel, normal)
            self.vel = self.vel - 2.0 * v_dot_n * normal
            
        elif other.mobile and not self.mobile:
            v_dot_n = np.dot(other.vel, normal)
            other.vel = other.vel - 2.0 * v_dot_n * normal
    


    def convert_to(self, new_type, system):
        """Convierte esta molécula a otro tipo simplemente para ver los colores bonitos u-u"""      

        if new_type in type_properties:
            props = type_properties[new_type]
            self.name = new_type
            self.color = props["color"]
            self.ratio = props["ratio"]
            self.active = props["active"]
            self.mobile = props["mobile"]

In [110]:
## SIMULACIÓN DE FOTOTRANSDUCCIÓN ##
class PhototransductionSystem:
    def __init__(self, L, verbose=False):
        self.molecules = []
        self.L = L
        self.center = np.array([L/2, L/2])
        self.R = nanodomin_space  
        self.current_time = 0.0
        
        self.verbose = verbose
        
        self.metrics_history = []
        self.rh_g_collisions = []
        self.g_pde_collisions = []
        self.pde_gmpc_collisions = []
        self.current_history = []   
        
        # Estado "macro" de la cascada / corriente
        self.current_cGMP = -0.2
        self.current_Ca = 0.2
        self.current_amplitude = 20
        self.initial_cGMP = C     
        self.gmpc_hydrolyzed_count = 0
        self.hydrolysis_events = []   

        self.phase = 1
        # Contadores y objetivos (80% efectivo)
        self.total_G = 0
        self.total_PDE = 0
        self.target_G_activated = None
        self.target_PDE_activated = None
        self.target_cGMP_hydrolysis = None
        
        self.rh_g_success = 0
        self.g_pde_success = 0
        
        # Para desactivación "efectiva" de PDE* en fase 4
        self.PDE_activity_factor = 1.0
        
        # Para NCKX retardado
        self.prev_I_ex = 0.0


    def check_boundary_hydrolysis(self):
        """
        Hidrólisis de GMPc cuando PDE* alcanza el borde.
        SOLO se activa en phase == 3.
        """
        if self.phase != 3:
            return 0

        hydrolysis_count = 0
        pde_active = [m for m in self.molecules if m.name == "PDE*"]
        
        if not pde_active:
            return 0

        base_prob = 0.8  
        concentration_factor = min(1.0, len(pde_active) / 2)
        
        for pde in pde_active:
            dist = np.linalg.norm(pde.pos - self.center)
            if dist >= 0.7 * self.R: 
                if np.random.random() < base_prob * concentration_factor:
                    gmps = [m for m in self.molecules if m.name == "GMPc"]
                    if gmps:
                        victim = random.choice(gmps)
                        self.molecules.remove(victim)
                        hydrolysis_count += 1
                        self.gmpc_hydrolyzed_count += 1

        return hydrolysis_count

    
    def random_position_in_disk(self):
        u = np.random.random()
        r = 0.9 * self.R * np.sqrt(u)  # Más cerca del centro
        theta = 2 * np.pi * np.random.random()
        x = self.center[0] + r * np.cos(theta)
        y = self.center[1] + r * np.sin(theta)
        return [x, y]

    def update_current(self, dt=0.05):
        """
        Dinámica de cGMP, Ca2+ y corriente de bastón.
        
        - cGMP: síntesis por GC (inhibida por Ca) e hidrólisis por PDE*.
        - Canal CNG: corriente catiónica rápida controlada por cGMP.
        - NCKX: extrusor de Ca2+ lento, dependiente de Ca (cola retardada).
        - Ca2+: entra por CNG, sale por NCKX.
        
        Convención de signos:
        - Corriente de CNG (I_photo) es NEGATIVA en oscuridad (entrada de cationes).
        - Cuando se cierra el canal (luz), la corriente se hace menos negativa → pulso hacia 0.
        """

        # ============================
        # 1. PARÁMETROS DEL MODELO
        # ============================

        # Canal CNG (dependiente de cGMP)
        n = 3.5         # cooperatividad
        K_cG = 2.5      # µM (semisaturación)
        I_CNG_max = 1.0 # amplitud máxima (unidades arbitrarias)

        # Fracción de Ca en la corriente CNG (el resto es Na+)
        frac_Ca = 0.15  # ~15% de la corriente es Ca2+

        # Intercambiador NCKX (extrusor de Ca2+)
        I_ex_max = 0.35   # amplitud máx NCKX
        K_ex = 0.3        # Ca para semisaturación
        tau_ex = 50.0     # ms, constante de tiempo del retardado

        # Síntesis / hidrólisis de cGMP
        k_hyd = 0.25      # hidrólisis por PDE activa
        k_dark = 0.05     # hidrólisis basal
        gc_max = 0.12     # síntesis máxima cuando Ca es muy bajo
        gc_half = 0.3     # Ca para inhibición de ciclasa
        gc_n = 4.0        # cooperatividad GCAP

        # Dinámica de Ca2+
        k_in = 0.015      # entrada de Ca por CNG
        k_out = 0.03      # salida por NCKX (escala I_ex)

        # ============================
        # 2. ESTADOS ACTUALES
        # ============================
        cG = self.current_cGMP
        Ca = self.current_Ca

        # Fracción de GMPc hidrolizado (para saber qué tan fuerte fue la cascada)
        total_GMPc = sum(1 for m in self.molecules if m.name == "GMPc")
        initial_GMPc = max(1, self.initial_cGMP)
        frac_hydro = 1.0 - (total_GMPc / initial_GMPc)

        # Fracción de PDE activa
        total_PDE = max(1, sum(1 for m in self.molecules if m.name.startswith("PDE")))
        active_PDE = sum(1 for m in self.molecules if m.name == "PDE*")
        fPDE_raw = active_PDE / total_PDE

        # Factor de actividad de PDE en la fase de desactivación (fase 4)
        if self.phase == 4:
            self.PDE_activity_factor *= 0.90
        else:
            self.PDE_activity_factor = 1.0

        fPDE = fPDE_raw * self.PDE_activity_factor

        # ============================
        # 3. DINÁMICA DE cGMP
        # ============================

        # Síntesis por GC inhibida por Ca (GCAP)
        gc_syn = gc_max / (1.0 + (Ca / gc_half)**gc_n)

        # Hidrólisis efectiva (oscuridad + PDE activa)
        hyd_effective = k_dark + k_hyd * fPDE

        # Ecuación diferencial para cGMP
        dcG = dt * (gc_syn - hyd_effective * cG)
        cG_new = max(cG + dcG, 1e-4)  # mantener positivo

        # Mientras haya fuerte activación (fase <=3 y mucha hidrólisis),
        # no permitimos que cGMP "rebote" hacia arriba artificialmente.
        if frac_hydro > 0.2 and self.phase <= 3 and cG_new > cG:
            cG_new = cG

        # ============================
        # 4. CANAL CNG (gating por cGMP)
        # ============================
        # Fracción abierta por Hill
        g_base = (cG_new**n) / (cG_new**n + K_cG**n + 1e-12)

        # Cierre extra cuando se ha hidrolizado suficiente GMPc (fase 1–3)
        if frac_hydro >= 0.33 and self.phase <= 3:
            g_CNG = g_base * 0.05
        else:
            g_CNG = g_base

        # Corriente total del canal CNG (inward, por convención negativa)
        I_CNG = -I_CNG_max * g_CNG

        # Descomponer en Na+ y Ca2+ (solo para tenerlos separados si luego quieres)
        I_CNG_Ca = I_CNG * frac_Ca
        I_CNG_Na = I_CNG * (1.0 - frac_Ca)

        # ============================
        # 5. NCKX RETARDADO
        # ============================
        # Corriente "objetivo" del intercambiador, dependiente de Ca
        I_ex_target = I_ex_max * (Ca / (K_ex + Ca + 1e-12))

        # Relajación exponencial hacia I_ex_target con tau_ex
        alpha = dt / (tau_ex + dt)
        I_ex = (1.0 - alpha) * self.prev_I_ex + alpha * I_ex_target
        self.prev_I_ex = I_ex

        # ============================
        # 6. DINÁMICA DE Ca2+
        # ============================
        # Ca entra por CNG (solo la parte Ca de la corriente) y sale por NCKX
        dCa = dt * (k_in * (-I_CNG_Ca) - k_out * I_ex)
        # (-I_CNG_Ca) porque I_CNG_Ca es negativa (entra Ca), queremos entrada positiva

        Ca_new = max(Ca + dCa, 1e-5)

        # ============================
        # 7. ACTUALIZAR ESTADOS Y CORRIENTE
        # ============================
        self.current_cGMP = cG_new
        self.current_Ca = Ca_new

        # Corriente "medida" por electrodo de succión:
        # normalmente lo que se ve es la corriente de CNG (fotocorriente).
        I_photo = I_CNG          # será negativa en oscuridad y sube hacia 0 con la luz
        I_total = I_photo        # si luego quieres sumar fugas u otras cosas, lo haces aquí

        self.current_history.append(I_total)



    def get_system_metrics(self):
        """Retorna métricas del sistema"""
        gmpc_count = sum(1 for mol in self.molecules if mol.name == "GMPc")
        current_val = self.current_history[-1] if self.current_history else 1.0

        return {
            'time': self.current_time,
            'GMPc_remaining': gmpc_count,
            'Rh_active': sum(1 for mol in self.molecules if mol.name == "Rh*"),
            'G_active': sum(1 for mol in self.molecules if mol.name == "G*"),
            'PDE_active': sum(1 for mol in self.molecules if mol.name == "PDE*"),
            'current': current_val,
            'total_molecules': len(self.molecules),
            'cGMP_concentration': self.current_cGMP,
            'Ca_concentration': self.current_Ca,
            'phase': self.phase
        }


    def update(self, dt=0.1):
        # ==========================================
        # 1. MOVIMIENTO + DESACTIVACIÓN ESTOCÁSTICA
        # ==========================================
        for molecule in self.molecules:
            molecule.move(self.L, dt)
            molecule.deactivate_step(dt)

        # ==========================================
        # 2. INTERACCIONES ENTRE MOLÉCULAS
        # ==========================================
        rh_g, g_pde, _ = self.check_all_interactions()

        # ==========================================
        # 3. HIDRÓLISIS POR PDE AL BORDE (SOLO UNA)
        # ==========================================
        pde_gmpc_hydrolysis = self.check_boundary_hydrolysis()

        # ==========================================
        # 4. REMOVER MOLÉCULAS MARCADAS
        # ==========================================
        if hasattr(self, "mark_for_removal") and self.mark_for_removal:
            for molecule in self.mark_for_removal:
                if molecule in self.molecules:
                    self.molecules.remove(molecule)
            self.mark_for_removal.clear()

        # ==========================================
        # 5. ACTUALIZAR CORRIENTE (UNA SOLA VEZ)
        # ==========================================
        self.update_current(dt)

        # ==========================================
        # 6. ACTUALIZAR TIEMPO (UNA SOLA VEZ)
        # ==========================================
        self.current_time += dt

        # ==========================================
        # 7. REGISTRAR MÉTRICAS
        # ==========================================
        metrics = self.get_system_metrics()
        self.metrics_history.append(metrics)

        self.rh_g_collisions.append(rh_g)
        self.g_pde_collisions.append(g_pde)
        self.pde_gmpc_collisions.append(pde_gmpc_hydrolysis)
        self.hydrolysis_events.append(pde_gmpc_hydrolysis)

        # ==========================================
        # 8. CONTROL DE FASES
        # ==========================================
        if self.phase == 1 and self.total_G > 0:
            if self.rh_g_success >= self.target_G_activated:
                self.phase = 2

        if self.phase == 2 and self.total_PDE > 0:
            if self.g_pde_success >= self.target_PDE_activated:
                self.phase = 3

        if self.phase == 3 and self.initial_cGMP > 0:
            total_GMPc = sum(1 for m in self.molecules if m.name == "GMPc")
            frac_hydro = 1.0 - (total_GMPc / self.initial_cGMP)
            if frac_hydro >= 0.50:
                self.phase = 4  # Desactivación + recuperación


        return rh_g, g_pde, pde_gmpc_hydrolysis, metrics['current']



    def molecules_match(self, mol1, mol2, type1, type2):
        return ((mol1.name == type1 and mol2.name == type2) or
                (mol1.name == type2 and mol2.name == type1))

    def check_all_interactions(self, interaction_range=2.0):
        """
        Fase 1: Rh* - G_inact  → G*
        Fase 2: G*  - PDE_inact → PDE*
        En fase 3 y 4 no hacemos colisiones entre proteínas (solo PDE-borde).
        """
        cell_size = interaction_range
        grid = {}

        for i, mol in enumerate(self.molecules):
            cx, cy = int(mol.pos[0] // cell_size), int(mol.pos[1] // cell_size)
            grid.setdefault((cx, cy), []).append(i)

        rh_g_count = 0
        g_pde_count = 0
        checked = set()

        for (cx, cy), indices in grid.items():
            neighbors = [(cx+dx, cy+dy) for dx in (-1,0,1) for dy in (-1,0,1)]
            neighbor_indices = [j for nb in neighbors for j in grid.get(nb, [])]

            for i in indices:
                mol1 = self.molecules[i]

                for j in neighbor_indices:
                    if i >= j or (i, j) in checked:
                        continue

                    mol2 = self.molecules[j]
                    distance = np.linalg.norm(mol1.pos - mol2.pos)

                    if self.phase == 1:# FASE 1: Rh* - G_inact
                        if self.molecules_match(mol1, mol2, "Rh*", "G_inact"):
                            if distance < 1.8:
                                rh_g_count += 1
                                self.rh_g_success += 1
                                if mol1.name == "G_inact":
                                    mol1.convert_to("G*", self)
                                else:
                                    mol2.convert_to("G*", self)

                                checked.add((i, j))
                                continue

                    elif self.phase == 2:    # FASE 2: G* - PDE_inact
                        if self.molecules_match(mol1, mol2, "G*", "PDE_inact"):
                            if distance < 1.6:
                                g_pde_count += 1
                                self.g_pde_success += 1
                                if mol1.name == "PDE_inact":
                                    mol1.convert_to("PDE*", self)
                                else:
                                    mol2.convert_to("PDE*", self)

                                checked.add((i, j))
                                continue

                    checked.add((i, j))

        return rh_g_count, g_pde_count, 0



    def random_position_outside_disk(self):
        """Posición fuera del disco de fototransducción"""
        for _ in range(100):
            x = np.random.uniform(0, self.L)
            y = np.random.uniform(0, self.L)
            dx = x - self.center[0]
            dy = y - self.center[1]
            dist = np.sqrt(dx**2 + dy**2)
            if dist > self.R:  # Fuera del disco
                return [x, y]
        return [self.L * 0.8, self.L * 0.8]  # Fallback

    def initialize_system(self):
        """Inicializa el sistema con las constantes definidas"""
        self.molecules.clear()

        # Rh* activa en el centro
        self.molecules.append(Molecule(
            name="Rh*", 
            pos=[self.L/2, self.L/2],
            vel=[random.uniform(-1, 1), random.uniform(-1, 1)],
            color="purple", ratio_molecular=0.3,
            energy=10, mass=2, active=True, mobile=True
        ))

        # Rh inactivas en el disco
        for _ in range(r-1):  
            self.molecules.append(Molecule(
                name="Rh_inact",
                pos=self.random_position_in_disk(),
                vel=[0, 0],
                color="violet", ratio_molecular=0.3,
                energy=0, mass=2, active=False, mobile=True
            ))

        # Proteínas G en el disco
        for _ in range(G):
            self.molecules.append(Molecule(
                name="G_inact",
                pos=self.random_position_in_disk(),
                vel=[random.uniform(-0.5, 0.5), random.uniform(-0.5, 0.5)],
                color="lightblue", ratio_molecular=0.2,
                energy=0, mass=1.5, mobile=True
            ))

        # PDE inactivas en el disco
        for _ in range(E):
            self.molecules.append(Molecule(
                name="PDE_inact",
                pos=self.random_position_in_disk(),
                vel=[0, 0],
                color="pink", ratio_molecular=0.25,
                energy=0, mass=2, mobile=True
            ))

        # GMPc fuera del disco
        for _ in range(C): 
            self.molecules.append(Molecule(
                name="GMPc",
                pos=self.random_position_outside_disk(),
                vel=[random.uniform(-1, 1), random.uniform(-1, 1)],
                color="white", ratio_molecular=0.1,
                energy=0, mass=1, mobile=True
            ))

        # Reset de historiales y estados
        self.metrics_history = []
        self.rh_g_collisions = []
        self.g_pde_collisions = []
        self.pde_gmpc_collisions = []
        self.current_history = []
        self.current_time = 0.0
        self.mark_for_removal = []
        
        self.current_cGMP = 4.0
        self.current_Ca = 0.5
        self.active_PDE_fraction = 0.0
        

        self.total_G = G
        self.total_PDE = E
        self.initial_cGMP = C
        
        # Objetivos fisiológicamente razonables
        self.target_G_activated = min(int(0.8 * self.total_G), 600)  
        self.target_PDE_activated = min(int(0.8 * self.total_PDE), 300)  
        self.target_cGMP_hydrolysis = int(0.5 * self.initial_cGMP)  
        
        self.rh_g_success = 0
        self.g_pde_success = 0
        self.gmpc_hydrolyzed_count = 0
        
        self.phase = 1
        self.PDE_activity_factor = 1.0
        self.prev_I_ex = 0.0
        
        # Inicializar corriente (estado oscuro)
        self.update_current(-0.5)


In [111]:
def run_complete_experiment(L, dt, seed=42, max_steps=20000):
    """
    Ejecuta un experimento completo para tamaño L.
    Corre hasta que la cascada vuelve ~a corriente oscura
    y devuelve también los tiempos de cambio de fase.
    """
    random.seed(seed)
    np.random.seed(seed)
    
    system = PhototransductionSystem(L)
    system.initialize_system()
    
    times = []
    rh_g_data = []
    g_pde_data = []
    pde_gmpc_data = []
    current_data = []

    t_f1 = None;     t_f2 = None ;      t_f3 = None ;     t_end = None 
    
    prev_phase = system.phase
    
    for _ in range(max_steps):
        rh_g, g_pde, pde_gmpc, current = system.update(dt)
        
        times.append(system.current_time)
        rh_g_data.append(rh_g)
        g_pde_data.append(g_pde)
        pde_gmpc_data.append(pde_gmpc)
        current_data.append(current)
        
        # Detectar cambios de fase
        new_phase = system.phase
        
        if prev_phase == 1 and new_phase == 2 and t_f1 is None:
            t_f1 = system.current_time
        if prev_phase == 2 and new_phase == 3 and t_f2 is None:
            t_f2 = system.current_time
        if prev_phase == 3 and new_phase == 4 and t_f3 is None:
            t_f3 = system.current_time
        
        prev_phase = new_phase
        
        # Condición fisiológica de fin: fase 4 y cGMP recuperado
        if (system.phase == 4 and 
            system.current_cGMP >= 0.95 * system.initial_cGMP):
            t_end = system.current_time
            break
    
    # Si alguna fase no se alcanzó,  usamos el tiempo final alcanzado
    if t_f1 is None:
        t_f1 = system.current_time
    if t_f2 is None:
        t_f2 = system.current_time
    if t_f3 is None:
        t_f3 = system.current_time
    if t_end is None:
        t_end = system.current_time
    
    return (
        np.array(times),
        np.array(rh_g_data),
        np.array(g_pde_data),
        np.array(pde_gmpc_data),
        np.array(current_data),
        t_f1, t_f2, t_f3, t_end
    )



In [112]:
def plot_complete_analysis(
    radios=[nanodomin_space, 0.5*nanodomin_space, 1.2*nanodomin_space],
    dt=1
):
    """3 paneles pequeños a la izquierda (fases) y panel grande de corriente a la derecha.
       Fases en tiempo REAL (Opción A)."""

    # Colores y etiquetas
    colors = ["#D6259B", "#51D63F", "#2580D6"]
    labels = [f'R = {r}' for r in radios]

    # Ejecutar todas las simulaciones
    all_data = {}
    Tend_max = 0  # tiempo total máximo entre radios

    for radio in radios:
        L = radio * 2
        (times, rh_g, g_pde, pde_gmpc, current,
         t_f1, t_f2, t_f3, t_end) = run_complete_experiment(L, dt=dt, seed=int(100+r))

        all_data[radio] = {
            "times": times,
            "rh_g": rh_g,
            "g_pde": g_pde,
            "pde_gmpc": pde_gmpc,
            "current": current,
            "t_f1": t_f1,
            "t_f2": t_f2,
            "t_f3": t_f3,
            "t_end": t_end
        }

        Tend_max = max(Tend_max, t_end)


    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(
        3, 2,
        width_ratios=[1, 2.3],  
        height_ratios=[1, 1, 1],
        hspace=0.28, wspace=0.25
    )

    ax_F1 = fig.add_subplot(gs[0, 0])
    ax_F2 = fig.add_subplot(gs[1, 0])
    ax_F3 = fig.add_subplot(gs[2, 0])
    ax_pulse = fig.add_subplot(gs[:, 1])  

    # ======================================================
    # PANEL GRANDE — CORRIENTE
    # ======================================================
    for i, radio in enumerate(radios):
        d = all_data[radio]
        times = d["times"]
        current = d["current"]

        ax_pulse.plot(times, current, color=colors[i], lw=2, label=labels[i])

    ax_pulse.set_title("A) Pulso de Corriente (Tiempo Real)", fontsize=18, fontweight="bold")
    ax_pulse.set_xlabel("Tiempo (ms)")
    ax_pulse.set_ylabel("Corriente Normalizada")
    ax_pulse.grid(alpha=0.3)
    ax_pulse.set_xlim(0,1500)
    ax_pulse.legend()

    # ======================================================
    # FASE 1 — Rh* → G*
    # ======================================================
    for i, radio in enumerate(radios):
        d = all_data[radio]
        times = d["times"]
        rhg = d["rh_g"]
        t_f1 = d["t_f1"]

        # tomar datos entre 0 y t_f1 (TIEMPO REAL)
        mask = (times >= 0) & (times <= t_f1)
        t_phase = times[mask]
        y_phase = np.cumsum(rhg[mask])

        ax_F1.step(t_phase, y_phase, color=colors[i], lw=2)

    ax_F1.set_title("B) Fase 1 — Rh* → G* (80%)", fontweight="bold")
    ax_F1.set_xlabel("Tiempo (ms)")
    ax_F1.set_ylabel("Colisiones")
    ax_F1.grid(alpha=0.3)

    # ======================================================
    # FASE 2 — G* → PDE*
    # ======================================================
    for i, radio in enumerate(radios):
        d = all_data[radio]
        times = d["times"]
        gpde = d["g_pde"]
        t_f1 = d["t_f1"]
        t_f2 = d["t_f2"]

        mask = (times >= t_f1) & (times <= t_f2)
        t_phase = times[mask]
        y_phase = np.cumsum(gpde[mask])

        ax_F2.step(t_phase, y_phase, color=colors[i], lw=2)

    ax_F2.set_title("C) Fase 2 — G* → PDE* (80%)", fontweight="bold")
    ax_F2.set_xlabel("Tiempo (ms)")
    ax_F2.set_ylabel("Colisiones")
    ax_F2.grid(alpha=0.3)

    # ======================================================
    # FASE 3 — PDE* → Hidrólisis
    # ======================================================
    for i, radio in enumerate(radios):
        d = all_data[radio]
        times = d["times"]
        pdeh = d["pde_gmpc"]
        t_f2 = d["t_f2"]
        t_f3 = d["t_f3"]

        mask = (times >= t_f2) & (times <= t_f3)
        t_phase = times[mask]
        y_phase = np.cumsum(pdeh[mask])

        ax_F3.step(t_phase, y_phase, color=colors[i], lw=2)

    ax_F3.set_title("D) Fase 3 — PDE* → Hidrólisis (80%)", fontweight="bold")
    ax_F3.set_xlabel("Tiempo (ms)")
    ax_F3.set_ylabel("Eventos")
    ax_F3.grid(alpha=0.3)

    plt.show()

plot_complete_analysis()

KeyboardInterrupt: 

In [ ]:
def run_multiple_simulations(n_simulations=3, L=None, dt=1):
    """Corre n simulaciones completas y devuelve:
       - curvas individuales
       - promedios por tiempo (corriente)
       - promedios por fase REAL (Rh→G, G→PDE, PDE→Hidrólisis)
       - tiempos promedio de fin de fase
    """

    if L is None:
        L = 2.0 * nanodomin_space

    all_runs = []

    print(f"Ejecutando {n_simulations} simulaciones...")

    for sim in range(n_simulations):
        print(f"  > Simulación {sim+1}/{n_simulations}")

        seed = 2024 + sim * 17

        (
            times, 
            rh_g, 
            g_pde, 
            pde_gmpc, 
            current,
            t_f1, t_f2, t_f3, t_end
        ) = run_complete_experiment(L, dt=dt, seed=seed)

        all_runs.append({
            "times": times,
            "rh_g": rh_g,
            "g_pde": g_pde,
            "pde_gmpc": pde_gmpc,
            "current": current,
            "t_f1": t_f1,
            "t_f2": t_f2,
            "t_f3": t_f3,
            "t_end": t_end
        })

    # ==== promedio de tiempos de fase ====
    mean_t_f1 = np.mean([r["t_f1"] for r in all_runs])
    mean_t_f2 = np.mean([r["t_f2"] for r in all_runs])
    mean_t_f3 = np.mean([r["t_f3"] for r in all_runs])
    mean_t_end = np.mean([r["t_end"] for r in all_runs])

    return {
        "runs": all_runs,
        "t_f1_avg": mean_t_f1,
        "t_f2_avg": mean_t_f2,
        "t_f3_avg": mean_t_f3,
        "t_end_avg": mean_t_end
    }


In [ ]:
def plot_complete_analysis(radios, dt=1, n_sim=4):
    """Panel derecho grande = corriente promedio.
       Paneles izquierdos = colisiones por fase REAL.
    """

    colors = ["#D6259B", "#51D63F", "#2580D6"]
    labels = [f"R={r}" for r in radios]

    # Contenedores por radio
    results = {}

    # === correr simulaciones para cada radio ===
    for radio in radios:
        print(f"\n=== Radio {radio} ===")
        res = run_multiple_simulations(n_simulations=n_sim, L=2*radio, dt=dt)
        results[radio] = res

    # === FIGURA ===
    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(3, 2, width_ratios=[1, 2.5], hspace=0.3, wspace=0.25)

    ax_F1 = fig.add_subplot(gs[0, 0])
    ax_F2 = fig.add_subplot(gs[1, 0])
    ax_F3 = fig.add_subplot(gs[2, 0])
    ax_pulse = fig.add_subplot(gs[:, 1])

    # ==================================================
    # PANEL A — CORRIENTE PROMEDIO (derecha)
    # ==================================================
    for c, radio in zip(colors, radios):
        runs = results[radio]["runs"]

        # unificar tiempos de cada corrida interpolando al tiempo máximo
        Tend = max(r["t_end"] for r in runs)
        t_plot = np.linspace(0, Tend, 800)

        currents_interp = []
        for r in runs:
            currents_interp.append(
                np.interp(t_plot, r["times"], r["current"])
            )

        currents_interp = np.array(currents_interp)

        mean_c = np.mean(currents_interp, axis=0)
        std_c = np.std(currents_interp, axis=0)

        ax_pulse.plot(t_plot, mean_c, lw=2, color=c, label=f"R={radio}")
        ax_pulse.fill_between(t_plot, mean_c-std_c, mean_c+std_c, color=c, alpha=0.20)

    ax_pulse.set_title("A) Pulso de Corriente (Promedio)", fontsize=17, fontweight="bold")
    ax_pulse.set_xlabel("Tiempo (ms)")
    ax_pulse.set_ylabel("Corriente Normalizada")
    ax_pulse.grid(alpha=0.3)
    ax_pulse.legend()

    # ==================================================
    # PANEL B — Fase 1 (tiempo real)
    # ==================================================
    for c, radio in zip(colors, radios):
        res = results[radio]
        t_f1_avg = res["t_f1_avg"]

        # tiempo real: 0 → t_f1_avg
        t_plot = np.linspace(0, t_f1_avg, 400)

        rh_interp = []
        for r in res["runs"]:
            mask = (r["times"] <= r["t_f1"])
            t_seg = r["times"][mask]
            y_seg = np.cumsum(r["rh_g"][mask])
            rh_interp.append(
                np.interp(t_plot, t_seg, y_seg)
            )

        rh_interp = np.array(rh_interp)
        m = np.mean(rh_interp, axis=0)
        s = np.std(rh_interp, axis=0)

        ax_F1.plot(t_plot, m, lw=2, color=c)
        ax_F1.fill_between(t_plot, m-s, m+s, color=c, alpha=0.2)

    ax_F1.set_title("B) Fase 1 — Rh*→G* (Tiempo real)", fontweight="bold")
    ax_F1.set_ylabel("Colisiones")
    ax_F1.grid(alpha=0.3)

    # ==================================================
    # PANEL C — Fase 2 (tiempo real)
    # ==================================================
    for c, radio in zip(colors, radios):
        res = results[radio]

        t_f1_avg = res["t_f1_avg"]
        t_f2_avg = res["t_f2_avg"]

        t_plot = np.linspace(t_f1_avg, t_f2_avg, 400)

        gp_interp = []
        for r in res["runs"]:
            mask = (r["times"] >= r["t_f1"]) & (r["times"] <= r["t_f2"])
            t_seg = r["times"][mask]
            y_seg = np.cumsum(r["g_pde"][mask])
            gp_interp.append(
                np.interp(t_plot, t_seg, y_seg)
            )

        gp_interp = np.array(gp_interp)
        m = np.mean(gp_interp, axis=0)
        s = np.std(gp_interp, axis=0)

        ax_F2.plot(t_plot, m, lw=2, color=c)
        ax_F2.fill_between(t_plot, m-s, m+s, color=c, alpha=0.2)

    ax_F2.set_title("C) Fase 2 — G*→PDE*", fontweight="bold")
    ax_F2.set_ylabel("Colisiones")
    ax_F2.grid(alpha=0.3)

    # ==================================================
    # PANEL D — Fase 3 (tiempo real)
    # ==================================================
    for c, radio in zip(colors, radios):
        res = results[radio]

        t_f2_avg = res["t_f2_avg"]
        t_f3_avg = res["t_f3_avg"]

        t_plot = np.linspace(t_f2_avg, t_f3_avg, 400)

        ph_interp = []
        for r in res["runs"]:
            mask = (r["times"] >= r["t_f2"]) & (r["times"] <= r["t_f3"])
            t_seg = r["times"][mask]
            y_seg = np.cumsum(r["pde_gmpc"][mask])
            ph_interp.append(
                np.interp(t_plot, t_seg, y_seg)
            )

        ph_interp = np.array(ph_interp)
        m = np.mean(ph_interp, axis=0)
        s = np.std(ph_interp, axis=0)

        ax_F3.plot(t_plot, m, lw=2, color=c)
        ax_F3.fill_between(t_plot, m-s, m+s, color=c, alpha=0.2)

    ax_F3.set_title("D) Fase 3 — PDE*→Hidrólisis", fontweight="bold")
    ax_F3.set_xlabel("Tiempo (ms)")
    ax_F3.set_ylabel("Eventos")
    ax_F3.grid(alpha=0.3)

    plt.show()


In [ ]:
plot_complete_analysis(radios=[1.0*nanodomin_space, 0.8*nanodomin_space, 1.2*nanodomin_space])



=== Radio 10.0 ===
Ejecutando 4 simulaciones...
  > Simulación 1/4
  > Simulación 2/4


KeyboardInterrupt: 